In [ ]:
import os
os.chdir("..")

In [2]:
import polars as pl
from src.data.data_process import cleanData
import ibis 

ibis.options.interactive = True

In [3]:
d = cleanData()
unique_df = pl.read_excel("data/processed/Query1_hac.xlsx")
# df.filter(df.id.isnull())
unique_df
#d.conn.insert("hactable", unique_df.cast(pl.String))

SEM_NUM_PAT,SEM_HSEG_SOC,SEM_FORMA_LEGAL,SEM_NOMBRE,SEM_COD_ACTIVOS,SEM_SIC,SEM_INDUSTRIA,SEM_SIC_NEW,SEM_INDUSTRIA_N,NAICS_R02,SEM_INDUS_R02
str,str,str,str,str,str,str,str,str,str,str
"""000""","""660432627""","""""","""Test1""","""""","""""","""""","""""","""""","""""",""""""
"""0000000003""","""660182772""","""1""","""CARIBBEAN BROADCASTING""","""*""","""4832""","""48310""","""4832""","""48310""","""515120""","""5151000"""
"""0000000003""","""660182272""","""1""","""CARIBBEAN BROADCASTING""","""*""","""4832""","""48310""","""4832""","""48310""","""515120""","""5151000"""
"""0000000005""","""660177110""","""1""","""MASCARO PORTER Y CO INC""","""*""","""5013""","""51012""","""5013""","""51012""","""423130""","""4231000"""
"""0000000010""","""660176270""","""1""","""E FRANCO Y CO INC""","""*""","""2051""","""20510""","""2051""","""20510""","""311812""","""3118000"""
…,…,…,…,…,…,…,…,…,…,…
"""0000865338""","""660326210""","""""","""ASOC MERCEDES MORALES""","""""","""""","""""","""""","""""","""""",""""""
"""000295837""","""660530699""","""""","""TEMP NETWORK INC""","""""","""""","""""","""""","""""","""""",""""""
"""000297739""","""660506875""","""""","""""","""""","""""","""""","""""","""""","""""",""""""


In [4]:
# ---------- Master CSV ----------

df = d.conn.table("qcewtable")
df = df.mutate(
  total_employment=(df.first_month_employment + df.second_month_employment + df.third_month_employment
)/3)

df2 = df


In [5]:
df = df2
df = df.mutate(
  first_4_naics_code=df.naics_code.cast("string").substr(0, 4),
  qtr=df.qtr.cast("int64"),
  total_wages=df.total_wages.cast("int64"),
  dummy=ibis.literal(1)
)


# Group by the specified columns and aggregate
df = (
  df.group_by(["year", "qtr", "first_4_naics_code"]).aggregate(
  total_wages_sum=df.total_wages.sum(),
  total_employment_sum=df.total_employment.sum(),
  dummy_sum=df.dummy.sum()
  )
  .order_by("year")
)

df = df.filter(df["dummy_sum"] > 4)

# Step 2: Add calculated columns for contributions
df = df.mutate(
  fondo_contributions=df["total_wages_sum"] * 0.014,
  medicare_contributions=df["total_wages_sum"] * 0.0145,
  ssn_contributions=df["total_wages_sum"] * 0.062
)

df_qcew = df
df_qcew



┏━━━━━━━┳━━━━━━━┳━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ year  ┃ qtr   ┃ first_4_naics_code ┃ total_wages_sum ┃ total_employment_sum ┃ dummy_sum ┃ fondo_contributions ┃ medicare_contributions ┃ ssn_contributions ┃
┡━━━━━━━╇━━━━━━━╇━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ int32 │ int64 │ string             │ int64           │ float64              │ int64     │ float64             │ float64                │ float64           │
├───────┼───────┼────────────────────┼─────────────────┼──────────────────────┼───────────┼─────────────────────┼────────────────────────┼───────────────────┤
│     0 │     1 │ ~                  │            NULL │                 NULL │         8 │                NULL │                   NULL │              NULL │
│     0 │     3 │ ~                  │            NULL │                 NULL │         8 │                NULL │                   NULL │              NULL │
│     1 │     3 │ ~                  │            NULL │                 NULL │         7 │                NULL │                   NULL │              NULL │
│     1 │     6 │ ~                  │            NULL │                 NULL │         8 │                NULL │                   NULL │              NULL │
│     1 │     7 │ ~                  │            NULL │                 NULL │         8 │                NULL │                   NULL │              NULL │
│     2 │     1 │ ~                  │            NULL │                 NULL │         8 │                NULL │                   NULL │              NULL │
│     5 │     2 │ ~                  │            NULL │                 NULL │         7 │                NULL │                   NULL │              NULL │
│     8 │     7 │ ~                  │            NULL │                 NULL │         8 │                NULL │                   NULL │              NULL │
│     9 │     2 │ ~                  │            NULL │                 NULL │         8 │                NULL │                   NULL │              NULL │
│    12 │     0 │ ~                  │            NULL │                 NULL │         8 │                NULL │                   NULL │              NULL │
│     … │     … │ …                  │               … │                    … │         … │                   … │                      … │                 … │
└───────┴───────┴────────────────────┴─────────────────┴──────────────────────┴───────────┴─────────────────────┴────────────────────────┴───────────────────┘

In [6]:
# ---------- Unique EXCEL ----------

# Load the data
df = d.conn.table("hactable")


df = df.mutate(
  first_4_naics_code=df.NAICS_R02.substr(0, 4)
)
df = df.filter(df.first_4_naics_code != "")

df2 = df.join(
  df_qcew, predicates=("first_4_naics_code")

)
# df2.to_csv("/jp-QCEW/data/processed/processed.csv")

df2

┏━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━┳━━━━━━━┳━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ id    ┃ SEM_NUM_PAT ┃ SEM_HSEG_SOC ┃ SEM_FORMA_LEGAL ┃ SEM_NOMBRE                 ┃ SEM_COD_ACTIVOS ┃ SEM_SIC ┃ SEM_INDUSTRIA ┃ SEM_SIC_NEW ┃ SEM_INDUSTRIA_N ┃ NAICS_R02 ┃ SEM_INDUS_R02 ┃ first_4_naics_code ┃ year  ┃ qtr   ┃ total_wages_sum ┃ total_employment_sum ┃ dummy_sum ┃ fondo_contributions ┃ medicare_contributions ┃ ssn_contributions ┃
┡━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━╇━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━╇━━━━━━━╇━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ int32 │ string      │ string       │ string          │ string                     │ string          │ string  │ string        │ string      │ string          │ string    │ string        │ string             │ int32 │ int64 │ int64           │ float64              │ int64     │ float64             │ float64                │ float64           │
├───────┼─────────────┼──────────────┼─────────────────┼────────────────────────────┼─────────────────┼─────────┼───────────────┼─────────────┼─────────────────┼───────────┼───────────────┼────────────────────┼───────┼───────┼─────────────────┼──────────────────────┼───────────┼─────────────────────┼────────────────────────┼───────────────────┤
│     2 │ 0000000003  │ 660182772    │ 1               │ CARIBBEAN BROADCASTING     │ *               │ 4832    │ 48310         │ 4832        │ 48310           │ 515120    │ 5151000       │ 5151               │  2005 │     1 │        18319554 │          2111.666667 │        80 │        2.564738e+05 │           2.656335e+05 │      1.135812e+06 │
│     3 │ 0000000003  │ 660182272    │ 1               │ CARIBBEAN BROADCASTING     │ *               │ 4832    │ 48310         │ 4832        │ 48310           │ 515120    │ 5151000       │ 5151               │  2005 │     1 │        18319554 │          2111.666667 │        80 │        2.564738e+05 │           2.656335e+05 │      1.135812e+06 │
│     4 │ 0000000005  │ 660177110    │ 1               │ MASCARO PORTER Y CO INC    │ *               │ 5013    │ 51012         │ 5013        │ 51012           │ 423130    │ 4231000       │ 4231               │  2005 │     1 │        14742881 │          1874.333333 │       132 │        2.064003e+05 │           2.137718e+05 │      9.140586e+05 │
│     5 │ 0000000010  │ 660176270    │ 1               │ E FRANCO Y CO INC          │ *               │ 2051    │ 20510         │ 2051        │ 20510           │ 311812    │ 3118000       │ 3118               │  2005 │     1 │        21233069 │          5795.000000 │       649 │        2.972630e+05 │           3.078795e+05 │      1.316450e+06 │
│     6 │ 0000000014  │ 660465359    │ 1               │ PACIFIC AGENCY INC         │ ~               │ 5031    │ 51030         │ 5031        │ 51030           │ 423310    │ 4233000       │ 4233               │  2006 │     2 │         5054310 │           787.000000 │        79 │        7.076034e+04 │           7.328750e+04 │      3.133672e+05 │
│     7 │ 0000000015  │ 660196234    │ 1               │ VOICE OF PUERTO RICO INC   │ *               │ 4832    │ 48310         │ 4832        │ 48310           │ 515111    │ 5151000       │ 5151               │  2005 │     1 │        18319554 │          2111.666667 │        80 │        2.564738e+05 │           2.656335e+05 │      1.135812e+06 │
│     8 │ 0000000017  │ 660189188    │ 8               │ FEMA                       │ ~               │ 9229    │ 90100         │ 9229        │ 90100           │ 922190    │ 9221000  

In [7]:
df2.to_csv("data/processed/processed.csv") gs

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))